In [37]:
import pandas as pd
import os
import openpyxl

In [38]:
# file_path = 'data/mini/NL RIVM-NEVO_mini.txt'
file_path = 'data/NL RIVM-NEVO 2008-05-22.txt'


with open(file_path, 'r', encoding='latin-1') as file:
    lines = file.readlines()

headers = lines[0].strip().split('\t')

data = []
for line in lines[1:]:
    fields = line.strip().split('\t')
    if len(fields) < len(headers):
        fields += [''] * (len(headers) - len(fields))
    data.append(fields)

# print(data)

df_nevo_langal = pd.DataFrame(data, columns=headers)

def convert_to_set(langual_codes):
    return set(langual_codes.split())

# Apply the function to the LANGUALCODES column
df_nevo_langal['LANGUALCODES'] = df_nevo_langal['LANGUALCODES'].apply(convert_to_set)

df_nevo_langal.head()

,FOODID,ORIGFDNM,ENGFDNAM,LANGUALCODES,REMARKS
0,945,Aardappel driekantjes bereid in diamantvet,Potatoes triangulardeep fat fried,"{A0152, A0829, B1218, A0706}",
1,947,Aardappel driekantjes bereid in oven,Potatoes triangular prepared in oven,"{A0152, A0829, B1218, A0706}",
2,946,Aardappel driekantjes bereid in zonnebloemolie,Potatoes triangular prep in sunflower oi,"{A0152, A0829, B1218, A0706}",
3,1457,Aardappelen gebakken,Potatoes fried,"{E0150, J0001, K0003, A0829, A0706, P0078, A01...",
4,982,Aardappelen gekookt,Potatoes boiled,"{E0150, J0001, K0003, A0829, P0037, A0706, G00...",


In [39]:
# Path to your Excel file
excel_path = 'data/Frida_5.1_November_2023.xlsx'

# Create a Pandas Excel file object
xls = pd.ExcelFile(excel_path)

# Load each sheet into a separate DataFrame
for sheet_name in xls.sheet_names:
    if sheet_name == "Food":
        df_frida_langal = pd.read_excel(xls, sheet_name)

# df_frida_langal

def convert_to_set(langual_codes):
    if isinstance(langual_codes, str):
        return set(langual_codes.split(','))
    else:
        return set()  # or np.nan if you prefer to keep NaN values

# # Apply the function to the LANGUALCODES column
df_frida_langal['LangualCode'] = df_frida_langal['LangualCode'].apply(convert_to_set)

df_frida_langal.head()

,FødevareNavn,FoodName,FoodID,TaxonomicName,NCBI,FoodEx2Code,FoodEx2Description,FoodOntology,LangualCode,FoodGroupID,FødevareGruppe,FoodGroup
0,"Jordbær, rå","Strawberry, raw",1,Fragaria x ananassa (Duchesne),3747.0,A01EA#F20.A07QT,"Strawberries, PART-CONSUMED-ANALYSED = W/o cap...",FOODON_03301722,"{B1393, E0150, A0707, G0003, A0833, H0003, K00...",51,Bærfrugt,Soft fruit
1,"Æble, uspec., råt","Apple, raw, all varieties",2,"Malus domestica (Borkhausen, 1803)",3750.0,A01DJ#F20.A07RH$F20.A07QH$F26.A07XD,"Apples, PART-CONSUMED-ANALYSED = W/o core, PAR...",FOODON_03301710,"{E0150, A0707, G0003, A0833, H0003, K0003, F00...",49,Kernefrugt,Pome fruit
2,"Banan, rå","Banana, raw",3,"Musa acuminata var. cavendish (Colla, 1820)",214697.0,A0DQM#F20.A07QF,"Common banana - acuminata cultivars, PART-CONS...",FOODON_03311513,"{E0150, A0707, G0003, A0833, H0003, K0003, F00...",52,Sydfrugt,Tropical or subtropical fruit
3,"Kartoffel, uspec., rå","Potato, raw",4,"Solanum tuberosum (Linnaeus, 1753)",4113.0,A00ZT#F20.A07QF,"Potatoes, PART-CONSUMED-ANALYSED = W/o peel",FOODON_03301449,"{E0150, A0650, G0003, H0003, K0003, A0829, F00...",39,Rod- og knoldgrøntsager,Root and tuber vegetables
4,"Sødmælk, konventionel (ikke-økologisk)","Milk, whole, konventional (not organic), 3.5 %...",6,"Bos taurus (Linnaeus, 1758)",9913.0,A02LY#F28.A07HV$F28.A07LJ$F21.A0C6Y$F07.A06ZY,"Cow milk, whole, PROCESS = Pasteurisation, PRO...",FOODON_03310780,"{N0017, E0123, H0306, A0626, A0724, G0003, A04...",4,Usyrnede mælkeprodukter,Unfermented milk products


In [40]:
# # file_path = 'data/mini/DK DTU_FDB 2008-06-26.txt'
# file_path = 'data/DK DTU_FDB 2008-06-26.txt'

# with open(file_path, 'r', encoding='latin-1') as file:
#     lines = file.readlines()

# headers = lines[0].strip().split('\t')

# data = []
# for line in lines[1:]:
#     fields = line.strip().split('\t')
#     if len(fields) < len(headers):
#         fields += [''] * (len(headers) - len(fields))
#     data.append(fields)

# df_frida_langal = pd.DataFrame(data, columns=headers)


# def convert_to_set(langual_codes):
#     return set(langual_codes.split())

# # Apply the function to the LANGUALCODES column
# df_frida_langal['LANGUALCODES'] = df_frida_langal['LANGUALCODES'].apply(convert_to_set)

# df_frida_langal

In [41]:
def jaccard_similarity(set1, set2):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union)

def langual_similarity(code1, code2):
    set1 = set(code1.split())
    set2 = set(code2.split())
    return jaccard_similarity(set1, set2)


In [42]:
def jaccard_similarity(set1, set2):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    # REMOVED; INACCURATE
    # return len(intersection) / len(union) 
    return len(intersection) / len(set1)
    

def langual_similarity(code1, code2):
    set1 = set(code1.split())
    set2 = set(code2.split())
    return jaccard_similarity(set1, set2)

# Calculate Jaccard similarity between the first row and the other rows
first_row_set = df_frida_langal['LangualCode'][0]
df_nevo_langal['SIMILARITYSCORE'] = df_nevo_langal['LANGUALCODES'].apply(lambda x: jaccard_similarity(first_row_set, x))
print(df_frida_langal['FoodName'][0])
print(df_frida_langal['LangualCode'][0])
# Set pandas options to display all rows
pd.set_option('display.max_rows', None)

# Sort the DataFrame based on SIMILARITY_TO_FIRST column from greatest to least
df_nevo_langal = df_nevo_langal.sort_values(by='SIMILARITYSCORE', ascending=False)

df_nevo_langal.head(50)

Strawberry, raw
{'B1393', 'E0150', 'A0707', 'G0003', 'A0833', 'H0003', 'K0003', 'F0003', 'A0732', 'P0024', 'J0003', 'Z0001', 'C0167', 'A0143', 'A0629', 'N0001', 'A0455', 'A0671', 'M0001'}


,FOODID,ORIGFDNM,ENGFDNAM,LANGUALCODES,REMARKS,SIMILARITYSCORE
14,148,Aardbeien,Strawberries,"{B1393, E0150, J0001, K0003, F0003, C0137, P00...",,0.631579
326,159,Cranberries vers,Cranberry,"{E0150, J0001, K0003, F0003, C0140, B1508, P00...",,0.578947
114,151,Banaan,Banana,"{E0150, J0001, K0003, F0003, C0138, P0024, A07...",,0.578947
218,157,Bramen,Blackberries,"{E0150, J0001, K0003, F0003, C0137, B1231, P00...",,0.578947
2060,1010,Vijgen vers,Figs fresh,"{E0150, J0001, K0003, F0003, C0140, P0024, A07...",,0.578947
1049,165,Mandarijn,Tangerines,"{E0150, J0001, K0003, F0003, P0024, A0707, B14...",,0.578947
391,160,Druiven witte/blauwe m schil,Grapes white/black with skin,"{E0150, J0001, K0003, F0003, C0137, P0024, A07...",,0.578947
28,150,Ananas,Pineapple,"{E0150, J0001, K0003, F0003, P0024, A0707, B14...",,0.578947
1431,169,Perzik z schil,Peach without skin,"{E0150, J0001, K0003, F0003, P0024, B1350, A07...",,0.578947
1413,168,Peer z schil,Pear without skin,"{E0150, B1344, J0001, K0003, F0003, P0024, A07...",,0.578947


In [46]:
# Calculate Jaccard similarity between the first row and the other rows
first_row_set = df_nevo_langal['LANGUALCODES'][0]
df_frida_langal['SIMILARITYSCORE'] = df_frida_langal['LangualCode'].apply(lambda x: jaccard_similarity(first_row_set, x))

# Sort the DataFrame based on SIMILARITY_TO_FIRST column from greatest to least
df_frida_langal = df_frida_langal.sort_values(by='SIMILARITYSCORE', ascending=False)

df_frida_langal.head(50)

,FødevareNavn,FoodName,FoodID,TaxonomicName,NCBI,FoodEx2Code,FoodEx2Description,FoodOntology,LangualCode,FoodGroupID,FødevareGruppe,FoodGroup,SIMILARITYSCORE
689,"Kartoffel, efterår (oktober til januar), rå","Potato, automn (October to January), raw",938,"Solanum tuberosum (Linnaeus, 1753)",4113.0,A00ZX#F20.A07QF,"Main-crop potatoes, PART-CONSUMED-ANALYSED = W...",FOODON_03301449,"{E0150, A0650, G0003, H0003, K0003, A0829, F00...",39,Rod- og knoldgrøntsager,Root and tuber vegetables,1.00
3,"Kartoffel, uspec., rå","Potato, raw",4,"Solanum tuberosum (Linnaeus, 1753)",4113.0,A00ZT#F20.A07QF,"Potatoes, PART-CONSUMED-ANALYSED = W/o peel",FOODON_03301449,"{E0150, A0650, G0003, H0003, K0003, A0829, F00...",39,Rod- og knoldgrøntsager,Root and tuber vegetables,1.00
697,"Kartoffel, gammel (februar - juni), rå","Potato, old (February to June), raw",946,"Solanum tuberosum (Linnaeus, 1753)",4113.0,A00ZX#F20.A07QF,"Main-crop potatoes, PART-CONSUMED-ANALYSED = W...",FOODON_03301449,"{E0150, A0650, G0003, H0003, K0003, A0829, F00...",39,Rod- og knoldgrøntsager,Root and tuber vegetables,1.00
616,"Kartoffel, ny (maj til september), rå","Potato, new (May to September), raw",846,"Solanum tuberosum (Linnaeus, 1753)",4113.0,A00ZV#F20.A07QF,"New potatoes, PART-CONSUMED-ANALYSED = W/o peel",FOODON_03301449,"{E0150, A0650, G0003, H0003, K0003, A0829, F00...",39,Rod- og knoldgrøntsager,Root and tuber vegetables,1.00
489,"Kartoffel, konserves","Potato, canned",691,"Solanum tuberosum (Linnaeus, 1753)",4113.0,A16FM#F27.A00ZT$F28.A07HZ$F06.A06XT$F22.A07SH,"Canned starchy root products, SOURCE-COMMODITI...",FOODON_03301885,"{J0123, G0003, C0240, E0151, H0001, M0135, A08...",46,Grøntsagskonserves,Canned vegetable products,0.75
581,"Kartoffel, kogt","Potato, boiled",801,"Solanum tuberosum (Linnaeus, 1753)",4113.0,A00ZT#F28.A07GL$F20.A07QF,"Potatoes, PROCESS = Boiling, PART-CONSUMED-ANA...",FOODON_03302197,"{E0150, G0015, A0650, H0003, K0003, A0829, A07...",39,Rod- og knoldgrøntsager,Root and tuber vegetables,0.75
907,"Pommes frites, friturestegt, fastfood","French fries, deepfried, fast food",1209,NaN,NaN,A0BYV#F28.A07GV,"French fries from cut potato, PROCESS = Deep f...",FOODON_03301959,"{C0240, K0003, H0367, A0830, P0024, E0152, J00...",127,Andre sammensatte fødevarer,Other composite foods,0.75
416,"Kartoffelmos, pulver uden tørmælk","Potatoes, mashed, instant powder without dry milk",602,NaN,NaN,A011E#F22.A07SH,"Mashed potato powder, PREPARATION-PRODUCTION-P...",FOODON_03307172,"{J0140, G0003, J0116, C0240, H0138, A0830, P00...",188,Andre grøntsagsprodukter,Other vegetable products,0.75
275,"Kartoffelmos, pulver med tørmælk","Potatoes, mashed, instant powder with dry milk",412,NaN,NaN,A011E#F22.A07SH,"Mashed potato powder, PREPARATION-PRODUCTION-P...",FOODON_03307172,"{J0140, G0003, J0116, C0240, K0003, H0138, A08...",188,Andre grøntsagsprodukter,Other vegetable products,0.75
615,"Yams, rå","Yam, raw",845,"Dioscorea (Linnaeus, 1753)",4672.0,A010D#F20.A07QF,"Yams, PART-CONSUMED-ANALYSED = W/o peel",FOODON_03307043,"{E0150, A0650, G0003, C0240, H0003, K0003, A08...",39,Rod- og knoldgrøntsager,Root and tuber vegetables,0.75
